## Construcción de nuevos predictores

In [ ]:
install.packages("fastDummies")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages("compare")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(dplyr)
library(forcats)
library(compare)


Attaching package: ‘compare’


The following object is masked from ‘package:base’:

    isTRUE




In [ ]:
dat <- read.csv("datos_ucaribe.csv", fileEncoding = "Latin1")

In [ ]:
nrow(dat)

[1] 16507

### Creación de variable dependiente (carga_aprobada)

Los posibles de la variable son: 
* 0: El estudiante ha reprobado al menos 1 asignatura de su carga académica
* 1: El estudiante ha aprobado toda su carga académica.

In [ ]:
matriculas <- unique(dat$matricula)
    dat$carga_aprobada <- 0
    for(mat in matriculas){
        carreras <- unique((dat %>% filter(matricula == mat))$carrera)
        
        for(car in carreras){ #en caso de que el estudiante haya cambiado de programa educativo
            periodos <- sort(unique((dat %>% filter(matricula == mat, carrera == car))$periodo))
            
            for(periodo in periodos){
                if(any(dat[dat$matricula== mat & dat$carrera == car & dat$periodo== periodo,][, "aprobado"] == 0)){
                    dat[dat$matricula==mat & dat$carrera == car & dat$periodo==periodo,][,"carga_aprobada"] <- 0
                }
                else{
                    dat[dat$matricula==mat & dat$carrera == car & dat$periodo==periodo,][,"carga_aprobada"] <- 1
                }
                
            }
        }
    }

* Variable binaria **recursando**

Indica si el estudiante está recursando alguna asignatura o no (0 = no está recursando, 1 = sí está recursando).

* Variable **total_recursando**

Total de asignaturas que está recursando en dicho periodo un estudiante.

*(Ambas se generan en la sig. celda de código).*

In [ ]:
matriculas <- unique(dat$matricula)
    dat$recursando <- 0
    dat$total_recursando <- 0
    for(mat in matriculas){
        carreras <- unique((dat %>% filter(matricula == mat))$carrera)
        
        for(car in carreras){ #en caso de que el estudiante haya cambiado de programa educativo
            periodos <- sort(unique((dat %>% filter(matricula == mat, carrera == car))$periodo))
            
            for(periodo in periodos){
                if(any(dat[dat$matricula== mat & dat$carrera == car & dat$periodo== periodo,][, "recursada"])){
                    dat[dat$matricula==mat & dat$carrera == car & dat$periodo==periodo,][,"recursando"] <- 1
                    #cantidad de asignaturas que está recursando en dicho semestre
                    num_recursadas <- sum(dat[dat$matricula== mat & dat$carrera == car & dat$periodo == periodo,]$recursada)
                    dat[dat$matricula==mat & dat$carrera == car & dat$periodo==periodo,][,"total_recursando"] <- num_recursadas
                    
                }
                else{
                    dat[dat$matricula==mat & dat$carrera == car & dat$periodo==periodo,][,"recursando"] <- 0
                    dat[dat$matricula==mat & dat$carrera == car & dat$periodo==periodo,][,"total_recursando"] <- 0
                }
                
            }
        }
    }

* Variable **depto**

Indica el departamento académico al que pertenece un estudiante.

In [ ]:
dat$depto <- ifelse(grepl("^Ingeniería",dat$carrera), "CBeI", 
                    ifelse(grepl("^Innovación|^Negocios",dat$carrera), "EyN", "TSGyH"))

* Variable binaria **practicante**

Indica si el estudiante está haciendo prácticas profesionales durante el semestre (1) o no (0).

In [ ]:
matriculas <- unique(dat$matricula)
    dat$practicante <- 0
    for(mat in matriculas){
        carreras <- unique((dat %>% filter(matricula == mat))$carrera)
        
        for(car in carreras){ #en caso de que el estudiante haya cambiado de programa educativo
            periodos <- sort(unique((dat %>% filter(matricula == mat, carrera == car))$periodo))
            
            for(periodo in periodos){
                if(any(grepl("^Prácticas profesionales", (dat[dat$matricula== mat & dat$carrera == car & dat$periodo == periodo,]$asignatura)))){
                    dat[dat$matricula==mat & dat$carrera == car & dat$periodo==periodo,][,"practicante"] <- 1
                }
                else{
                    dat[dat$matricula==mat & dat$carrera == car & dat$periodo==periodo,][,"practicante"] <- 0
                }
                
            }
        }
    }

## Reducir observaciones

Actualmente cada observación del conjunto de datos corresponde a una *asignatura* cursada por un estudiante en determinado periodo. Para predecir la variable **carga_aprobada** se necesita que cada observación corresponda a un *estudiante* por semestre. 

In [ ]:
estudiantes<- distinct(dat, matricula, carrera, periodo, .keep_all = TRUE)
#estudiantes <- subset(estudiantes, select = -c(clave, asignatura, docente, promediofinal, decil_asig, 
#                                                         quintil_asig, cuartil_asig, cuartil_docente, tasa_rep_doc, 
#                                                         tasa_rep_doc_asig, tasa_rep_asig, aprobado, recursada))

In [ ]:
nrow(estudiantes) #número de observaciones
length(unique(estudiantes$matricula)) #número de estudiantes

[1] 2805

[1] 532

* Variable categórica **situacion**


In [ ]:
estudiantes$situacion <- ""
# 1. Regular: numRepPrev == 0 & total_recursando == 0
estudiantes[estudiantes$numRepPrev==0 & estudiantes$total_recursando == 0,][,"situacion"] <- "Regular"

# 2. Irregular: numRepPrev > 0 & asigRep_totalAsig < 0.5 | (total_recursando > 0 & total_recursando <3) & cantidad_asig > 3
estudiantes[estudiantes$numRepPrev>0 & estudiantes$asigRep_totalAsig<0.5 | (estudiantes$total_recursando %in% c(1,2)) & estudiantes$cantidad_asig>3,][,"situacion"] <- "Irregular"

# 3. Condicionado: asigRep_totalAsig >= 0.5 & total_recursando >= 2 & cantidad_asig == 3
estudiantes[estudiantes$asigRep_totalAsig>=0.5 | estudiantes$total_recursando>=2 & estudiantes$cantidad_asig == 3,][,"situacion"] <- "Condicionado"

## Dummy variables

In [ ]:
library(fastDummies)

In [ ]:
# Convertir situacion a dummy variables
estudiantes <- dummy_cols(estudiantes, select_columns = "situacion", remove_first_dummy = TRUE)

In [ ]:
estudiantes <- dummy_cols(estudiantes, select_columns = "depto", remove_first_dummy = TRUE)

In [ ]:
length(names(estudiantes))
names(estudiantes)

[1] 48

[1] "matricula"              "carrera"                "periodo"               
 [4] "numRepPrev"             "clave"                  "año"                   
 [7] "semestre"               "recursada"              "cantidad_asig"         
[10] "tasa_rep_carga"         "tasa_rep_asig"          "cuartil_asig"          
[13] "quintil_asig"           "decil_asig"             "docente"               
[16] "asignatura"             "promediofinal"          "ceneval_global"        
[19] "ceneval_analitico"      "ceneval_matematico"     "ceneval_lengua"        
[22] "ceneval_esp"            "aprobado"               "prom_per_prev"         
[25] "tasa_rep_doc_asig"      "tasa_rep_doc"           "cuartil_docente"       
[28] "generacion"             "tasa_aprob_per_prev"    "tasa_aprob_gral_prev"  
[31] "asigMuchas"             "cuartil_repEst"         "asigRep_totalAsig"     
[34] "año_encurso"            "complejidad_carga4"     "complejidad_carga"     
[37] "complejidad_carga5"     "semestre_Otoño"         "carga_aprobada"        
[40] "recursando"             "total_recursando"       "depto"                 
[43] "practicante"            "situacion"              "situacion_Condicionado"
[46] "situacion_Irregular"    "situacion_Regular"      "depto_"

In [ ]:
estudiantes <- subset(estudiantes, select = -c(clave, docente, promediofinal, decil_asig, 
                                                          cuartil_asig, cuartil_docente, 
                                                          aprobado, depto,depto_))

In [ ]:
ncol(estudiantes)

[1] 39

In [ ]:
# Guardar nuevo conjunto de datos
#setwd("C:/Users/regina/Desktop/Analitica/proyectoterminal/carga_aprobada")

write.csv(estudiantes, "estudiantes.csv", row.names = FALSE)